In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Loading Files**

In [2]:
train_file = pd.read_csv("./kaggle/input/train.csv")
test_file = pd.read_csv("./kaggle/input/test.csv")
submission_file = pd.read_csv("./kaggle/input/sample_submission.csv")

In [3]:
train_file.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test_file.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Setting up X Y and cleaning**

In [5]:
X_train = train_file.drop(columns=['label']).values
Y_train = train_file['label'].values

X_test = test_file.values

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [6]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
Y_train

array([1, 0, 1, ..., 7, 6, 9])

Normally you would need to flatten images and then normalize the vectors between 0 or 1 but this dataset has this applied 

In [8]:
print(X_train.shape)

(42000, 784)


**Creating Model**

In [9]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))  # Input layer with 784 input units
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Output layer for 10 classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


/Users/joanmascastella/Desktop/PYTHON_ENV/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - accuracy: 0.8435 - loss: 0.5327 - val_accuracy: 0.9535 - val_loss: 0.1608
Epoch 2/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 884us/step - accuracy: 0.9607 - loss: 0.1337 - val_accuracy: 0.9620 - val_loss: 0.1269
Epoch 3/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 698us/step - accuracy: 0.9754 - loss: 0.0858 - val_accuracy: 0.9690 - val_loss: 0.1036
Epoch 4/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 716us/step - accuracy: 0.9791 - loss: 0.0639 - val_accuracy: 0.9701 - val_loss: 0.1061
Epoch 5/5
1050/1050 ━━━━━━━━━━━━━━━━━━━━ 1s 685us/step - accuracy: 0.9855 - loss: 0.0463 - val_accuracy: 0.9708 - val_loss: 0.1078


**Train and testing the network**

In [10]:
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=2)
scores = model.evaluate(X_train, Y_train, verbose=0)

Epoch 1/5
1050/1050 - 1s - 779us/step - accuracy: 0.9888 - loss: 0.0369 - val_accuracy: 0.9712 - val_loss: 0.1111
Epoch 2/5
1050/1050 - 1s - 703us/step - accuracy: 0.9890 - loss: 0.0349 - val_accuracy: 0.9714 - val_loss: 0.1122
Epoch 3/5
1050/1050 - 1s - 719us/step - accuracy: 0.9923 - loss: 0.0234 - val_accuracy: 0.9740 - val_loss: 0.0957
Epoch 4/5
1050/1050 - 1s - 708us/step - accuracy: 0.9922 - loss: 0.0239 - val_accuracy: 0.9679 - val_loss: 0.1273
Epoch 5/5
1050/1050 - 1s - 686us/step - accuracy: 0.9926 - loss: 0.0214 - val_accuracy: 0.9742 - val_loss: 0.1104


In [11]:
print('Accuracy: {}% \n Error: {}'.format(scores[1], 1 - scores[1]))        

Accuracy: 0.9917380809783936% 
 Error: 0.008261919021606445


Saving model

In [16]:
pretrained_model = keras.saving.save_model(model, 'classification.keras')

**Submission**

In [ ]:
proba=model.predict(X_test)
y_pred=np.argmax(proba,axis=1)
y_pred

In [14]:
submission_file["Label"]=y_pred
submission_file.to_csv("submission.csv",index=False)

In [ ]:
submission_file